In [ ]:
import pandas as pd
import boto3

In [ ]:
# Read data
data = pd.read_csv("../data/raw/train.csv")
print("data shape :", data.shape)
data.head(5)

data shape : (2018352, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [ ]:
# Seperate consumption and production data
consumption = data[data["is_consumption"]==1]
print("consumption data shape :", consumption.shape)
display(consumption.head())
production = data[data["is_consumption"]==0]
print("production data shape :", production.shape)
display(production.head())

consumption data shape : (1009176, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
5,0,0,3,656.859,1,2021-09-01 00:00:00,0,5,2
7,0,1,0,59.000,1,2021-09-01 00:00:00,0,7,3
9,0,1,1,501.760,1,2021-09-01 00:00:00,0,9,4


production data shape : (1009176, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
6,0,1,0,0.000,0,2021-09-01 00:00:00,0,6,3
8,0,1,1,0.000,0,2021-09-01 00:00:00,0,8,4


In [ ]:
len(data["data_block_id"].unique())

638

In [ ]:
# Keep only target values, datetime and block ID of consumption data
consumption = consumption.loc[:, ["datetime", "data_block_id", "target"]].rename(columns={"target": "consumption"})
print("consumption data shape :", consumption.shape)
consumption.head()

consumption data shape : (1009176, 3)


,datetime,data_block_id,consumption
1,2021-09-01 00:00:00,0,96.590
3,2021-09-01 00:00:00,0,17.314
5,2021-09-01 00:00:00,0,656.859
7,2021-09-01 00:00:00,0,59.000
9,2021-09-01 00:00:00,0,501.760


In [ ]:
# save data in ./data directory
consumption.to_csv("./data/consumption.csv", index=False)

In [ ]:
# Check data has been correctly saved
consumption_loaded = pd.read_csv("./data/consumption.csv")
print("consumption data shape :", consumption.shape)
consumption.head()

consumption data shape : (1009176, 3)


,datetime,prediction_unit_id,consumption
1,2021-09-01 00:00:00,0,96.590
3,2021-09-01 00:00:00,1,17.314
5,2021-09-01 00:00:00,2,656.859
7,2021-09-01 00:00:00,3,59.000
9,2021-09-01 00:00:00,4,501.760


In [ ]:
# Load data to AWS S3 (Optional)
s3_client = boto3.client('s3')
with open("./data/consumption.csv", "rb") as file:
    s3_client.upload_fileobj(file, "enefit-competition", "data/consumption.csv")

In [ ]:
# Check that data has been uploaded correctly
response = s3_client.list_objects(
    Bucket='enefit-competition')
for obj in response.get("Contents"):
    print(obj.get("Key"))

data/consumption.csv
